##CHANGELOG
- improved prompts for spymasters
- proper voting for llm teams
- prompting history of the game
- make the function more general (other llms like Claude, Grok etc. can play if play function is implemented; even let humans team play)

In [74]:
#@title import dependencies
!pip install openai
import requests
from tqdm import tqdm
import random
import re
import ast
from openai import OpenAI
from google.colab import userdata
from IPython.display import IFrame, display, HTML

## Utils Functions

In [57]:
prompt_colors = {
    "RED": "\033[1;31m",
    "BLUE": "\033[1;34m",
    "endcolor": "\033[0m",
    "n": "\033[1;33m"
  }

#BOARD GENERATOR
def generate_board(n = 25, lang = "eng", c = 7, k = 1):
  '''
  inputs:
    n (int): number of words
    lang (str): language of words
    c (int): number of colored words for each team
    k (int): number of killer words

  returns a dictionary such that dict[word] = color
  '''
  blue_cards = c
  red_cards = c + 1
  if (2*c + k + 1) > n:
    raise Exception("ERROR: wrong parameters for board creation")

  url = f"https://raw.githubusercontent.com/mich1803/Codenames-LLM/main/wordlists/{lang}.txt"

  try:
    # Fetch the content of the file
    response = requests.get(url)
  except:
    raise Exception("ERROR: The language entered is wrong or has no dictionary in the github folder")
  words = response.text.splitlines()

  # Select n unique random words
  random_words = random.sample(words, n)
  random_words = [word.upper() for word in random_words]

  # Select indices for the colors
  indices = list(range(n))
  blue_indices = random.sample(indices, blue_cards)
  remaining_indices = [i for i in indices if i not in blue_indices]
  red_indices = random.sample(remaining_indices, red_cards)
  remaining_indices = [i for i in remaining_indices if i not in red_indices]
  black_index = random.sample(remaining_indices, k)

  colors = [
      "BLUE" if i in blue_indices else
      "RED" if i in red_indices else
      "KILLER" if i in black_index else
      "NEUTRAL"
      for i in range(n)
      ]

  word_color_dict = {random_words[i]: colors[i] for i in range(n)}

  return word_color_dict

#BOARD FORMATTING FOR PROMPT
def board4prompt(word_color_dict, master = False):
  '''
  inputs:
    word_color_dict(dict): board dict generated from generate_board()

  return a string version for the prompt for LLMs
  '''

  text = "|"
  if master:
    for idx, i in enumerate(word_color_dict):
      text += f" {i} ({word_color_dict[i]}) " + "|"
      if (idx > 0) and (((idx + 1) % 5) == 0):
        text += "\n"
        if idx != (len(word_color_dict)-1):
          text += "|"
  else:
    for idx, i in enumerate(word_color_dict):
      text += f" {i} " + "|"
      if (idx > 0) and ((idx % 5) == 0):
        text += "\n"
        if idx != (len(word_color_dict)-1):
          text += "|"


  return text

#STYLISH BOARD FORMATTING FOR PRINT
def board4print(word_color_dict, master = False):
  '''
  inputs:
    word_color_dict(dict): board dict generated from generate_board()

  return a formatted string version for printing
  '''
  COLOR = {
      "BLUE": "blue",
      "RED": "red",
      "KILLER": "black",
      "NEUTRAL": "gray"
  }
  text = "<body style='display: flex; justify-content: center; align-items: center'><div style='display: grid; grid-template-columns: repeat(5, 1fr); gap: 10px; width: 60%;'>"
  end = "</div></body>"
  if not master:
    for word in word_color_dict:
      card = f"""
        <div style='
            font-size: 30px;
            display: flex;
            justify-content: center;
            align-items: center;
            width: 100%;
            padding-top: 15%;
            background-color: gray;
            border-radius: 10px;
            box-shadow: 0 2px 5px rgba(0, 0, 0, 0.2);'>
            <b style='margin: 0;'>{word}</b>
        </div>
        """
      text += card
  elif master:
    for word in word_color_dict:
      card = f"""
        <div style='
            font-size: 30px;
            display: flex;
            justify-content: center;
            align-items: center;
            width: 100%;
            padding-top: 15%;
            background-color: {COLOR[word_color_dict[word]]};
            border-radius: 10px;
            box-shadow: 0 2px 5px rgba(0, 0, 0, 0.2);'>
            <b style='margin: 0;'>{word}</b>
        </div>
        """
      text += card
  return text + end

In [58]:
board = generate_board(n = 25, lang = "eng", c = 7, k = 1)
display(HTML(board4print(board, master = True)))
display(HTML("<br><br>"))
display(HTML(board4print(board)))


## API unifier

In [59]:
def call_api(system_prompt, prompt, model, json_mode):

  #OPENAI models:
  if model == "gpt-4o-mini": #or other OpenAI models
    API_key = userdata.get('openai')
    client = OpenAI(api_key=API_key)

    #OPENAI normal mode
    if not json_mode:
      chat_completion = client.chat.completions.create(
            model=model,
            messages=[
                      {"role": "system", "content": system_prompt},
                      {"role": "user", "content": prompt}
                      ]
        )
      return chat_completion.choices[0].message.content

    #OPENAI json mode
    if json_mode:
      chat_completion = client.chat.completions.create(
        model=model,
        response_format={ "type": "json_object" },
        messages=[
                  {"role": "system", "content": system_prompt},
                  {"role": "user", "content": prompt}
                  ]
        )
      answer = chat_completion.choices[0].message.content
      #print(f"Raw API Response: {answer}")  # Print the raw response for debugging
      try:
        data = ast.literal_eval(answer)

      except (SyntaxError, ValueError) as e:
        # Catch specific errors and provide more context in the error message
        print(f"Error parsing API response: {e}")
        print(f"Response content: {answer}")
        raise ValueError("Format error: The API response is not a valid Python literal.") from e  # Chain exceptions for better debugging
      return data

    else:
      raise Exception("ERROR: wrong model name")


## Unify agents

In [96]:
def spymaster(lang, board, team, history, model):
  json_mode = True
  sys_prompt = (
    "We are playing the board game CODENAMES, and your role is the spymaster.\n"
    f"You are part of team {team}, and your job is to help your team guess the words associated with your team's color.\n"
    "You will be given a list of words, each with an associated color: RED for team red words, BLUE for team blue words, "
    "KILLER for killer words, and NEUTRAL for neutral words.\n"
    "Your goal is to provide a ONE-WORD CLUE and a NUMBER. The clue should help your team guess only the words with your team's color.\n"
    "If your team guesses the correct word, you gain 1 point. If your team guesses an opponent's word, they gain 1 point. "
    "If your team guesses the KILLER word, you lose instantly, so be extremely cautious!\n"
    "Be careful with your clues! DO NOT use any words that are already on the board, as that would immediately make you lose.\n"
    "Your clue should not be too vague or too specific. The clue should help your team make a guess about the words on the board, but it should be safe and avoid leading to mistakes.\n"
    "You should also be mindful of these important factors:\n"
    "1. Avoid using a word from the board as your clue. If you do, you will lose instantly.\n"
    "2. The clue should not be misleading or overly broad. Think of how it could be interpreted and avoid ambiguities.\n"
    "3. The number you provide should represent how many words on the board are related to your clue.\n"
    "4. The number should be chosen carefully. Do not overestimate how many words fit the clue, and avoid making the number too high, "
    "as it could increase the risk of the guessers making mistakes.\n"
    "5. If your team guesses the KILLER word, you will lose the game.\n"
    "6. Consider the current state of the game: If there are few words left, give more specific clues. If there are many words left, "
    "give broader clues that point to multiple words.\n"
    "After you choose the clue, double-check if it’s safe and strategically helpful for your team.\n"
    "YOUR RESPONSE SHOULD BE A JSON OBJECT WITH TWO KEYS: 'clue' and 'number'. Make sure the clue is a single word, and the number "
    "is an integer between 1 and the number of words left on the board related to that clue.\n"
)

  prompt = f"The board is {board4prompt(board, master = True)}, The history of the game is: {history}."

  response = call_api(sys_prompt, prompt, model, json_mode)
  return response["clue"].upper(), response["number"]

def guesser(lang, team, board, clue, n_guessers, i_agent, cards_remaining, idea, k, history, conv, model = "gpt-4o-mini"):
  opp = "BLUE" if team == "RED" else "RED"
  json_mode = True
  sys_prompt = "We are playing the board game CODENAMES, and your role" + \
              f"will be the guesser. \n You're in a team of {n_guessers}" +\
              f"other guessers. Your objective as a team is to guess" +\
              f"the words of your color based on the clue of your spymaster \n" +\
              f"There are: {cards_remaining[team]} words for your team, "+\
              f"{cards_remaining[opp]} words for the enemy team and {k} killer cards." + \
              f"Even if the number said by the spymaster is more than one, let's guess one word at a time. \n" +\
              f"You are in team {team}. You are agent number {i_agent}." +\
              f"Your inital thoughts were: {idea} \n" +\
              f"You are a very good JSON file writer." +\
              f"You need to give in output a dict with 3 keys:\n" +\
              f"M: your message to other teammates (be short and coincise); \n" +\
              f"W: a bool that is 1 if you want to listen and speak to the others again, 0 if you feel like you wouldn't add anything to the conversation (*if you all agree please say 0*); \n" +\
              f"V: a dictionary that maps each word in the board to a real number in iterval (-10,10) that represents your confidence in guessing that word (don't be afraid of writing decimal numbers)."

  prompt = f"The board is {board4prompt(board)}. \n" +\
          f"The history is {history}. \n" +\
          f"The clue is: {clue}. \n" +\
          f"The previous conversation is: {conv}."

  response = call_api(sys_prompt, prompt, model, json_mode)
  #print(response["W"])
  return response["M"], bool(response["W"]), response["V"]


def guesser_ideas(lang, team, board, clue, n_guessers, i_agent, cards_remaining, k, history,  model = "gpt-4o-mini"):
  opp = "BLUE" if team == "RED" else "RED"
  json_mode = False
  sys_prompt = "We are playing the board game CODENAMES, and your role" + \
              f"will be the guesser. \n You're in a team of {n_guessers}" +\
              f"other guessers. Your objective as a team is to guess" +\
              f"the words of your color based on the clue of your spymaster \n" +\
              f"There are: {cards_remaining[team]} words for your team, "+\
              f"{cards_remaining[opp]} words for the enemy team and {k} killer cards." + \
              f"Even if the number said by the spymaster is more than one, let's guess one word at a time. \n" +\
              f"You are in team {team}. You are agent number {i_agent}." +\
              "Try to say a max of 200 characters."

  prompt = f"The board is {board4prompt(board)}. \n" +\
          f"The history is {history}. \n" +\
          f"The clue is: {clue}. \n" +\
          "Share your (short and coincise) initial thoughts with your other fellow teammates."

  response = call_api(sys_prompt, prompt, model, json_mode)
  return response



## Vote system

In [61]:
def vote_system(votes):
  results = {}
  for vote in votes:
    for word, points in vote.items():
      if word in results:
        results[word] += points
      else:
        results[word] = points
  return max(results, key=results.get)

## Turno

In [98]:
def play_turn(lang, team, board, cards_remaining, k, n_guessers, history, model = "gpt-4o-mini", verbose = False, masterverbose = False):
  opp = "BLUE" if team == "RED" else "RED"
  rc = cards_remaining
  b = board

  if verbose:
    print("The actual board is: ")
    if masterverbose:
      display(HTML(board4print(board, master = True)))
    else:
      display(HTML(board4print(board)))
    print(f"it is {prompt_colors[team]}{team}{prompt_colors['endcolor']} team turn: \n")

  if model == "human":
    clue = input(f"Insert the clue for the {prompt_colors[team]}{team}{prompt_colors['endcolor']} team:").upper()
    number = input(f"Insert the number of words for the {prompt_colors[team]}{team}{prompt_colors['endcolor']} team:")
    spymaster_history = f"Team {team} spymaster said: {clue} ({number})."
    guesser_history = []

    if clue in b:
      return {"endgame": True, "win": opp, "reason": "spymaster said a word in the board"}

    for i in range(int(number)):
      guess = input(f"Insert the guess for the {prompt_colors[team]}{team}{prompt_colors['endcolor']} team:").upper()

  else:
    clue, number = spymaster(lang, board, team, history, model)
    spymaster_history = f"Team {team} spymaster said: {clue} ({number})."
    guesser_history = []
    if verbose: print(f"The {prompt_colors[team]}{team}{prompt_colors['endcolor']} spymaster's clue is: {clue} ({number}). \n \n")

    if clue in b:
      return {"endgame": True, "win": opp, "reason": "spymaster said a word in the board"}


    for i in range(number):
      ideas = []
      votes = [{} for _ in range(n_guessers)]
      want_to_talk = [True for _ in range(n_guessers)]
      conv = []
      for j in range(n_guessers):
        ideas.append(guesser_ideas(lang, team, board, clue, n_guessers, j, cards_remaining, k, history,  model))
        if verbose: print(f"Team {prompt_colors[team]}{team}{prompt_colors['endcolor']} guesser {j} thinks: {ideas[j]}.")
        conv.append(f"Team {team} guesser {j} said: {ideas[j]}.")
      speaktoomuch = 0
      while (any(want_to_talk)) and speaktoomuch < 2:
        if verbose: print(f"Do you want to talk? {want_to_talk}")
        for j in range(n_guessers):
          if want_to_talk[j]:
            mess, want_to_talk[j], votes[j] = guesser(lang, team, board, clue, n_guessers, j, cards_remaining, ideas[j], k, history, conv, model)
            if verbose: print(f"Team {prompt_colors[team]}{team}{prompt_colors['endcolor']} guesser {j} said: {mess}.")
            conv.append(f"Team {team} guesser {j} said: {mess}.")
        speaktoomuch += 1
      guess = vote_system(votes)
      if verbose: print(f"{prompt_colors['n']}NARRATOR{prompt_colors['endcolor']}: Team {prompt_colors[team]}{team}{prompt_colors['endcolor']} voted: {guess}. \n \n")


      try:
          x = b[guess]
      except:
          if verbose: print(f"{prompt_colors[team]}{team}{prompt_colors['endcolor']} team guess wasn't on the board. \n \n")
          guesser_history.append(f"Team {team} said: {guess}. The word was not in board")
          return {"endgame": False, "rc": rc, "b": b, "spyh": spymaster_history, "teamh": guesser_history}

      if x == "KILLER":
          if verbose: print(f"The killer word have been selected, the game ends. \n \n ")

          return {"endgame": True, "win": opp, "reason": "guessed killer word"}

      elif x == team:
          rc[team] -= 1
          if verbose: print(f'A {prompt_colors[team]}{team} word{prompt_colors["endcolor"]} have been selected ({team} cards remaining = {rc[team]}). \n \n')
          del b[guess]
          guesser_history.append(f"Team {team} said: {guess}. The word was {team}.")
          if rc[team] == 0:
              if verbose: print(f"The {prompt_colors[team]}{team} team{prompt_colors['endcolor']} reached the goal, the game ends. \n \n")
              return {"endgame": True, "win": team, "reason": "cards finished"}

      elif x == opp:
          rc[opp] -= 1
          if verbose: print(f'A {prompt_colors[opp]}{opp} word{prompt_colors["endcolor"]} have been selected ({opp} cards remaining = {rc[opp]}). \n \n')
          del b[guess]
          guesser_history.append(f"Team {team} said: {guess}. The word was {opp}.")
          if rc[opp] == 0:
              if verbose: print(f"The {prompt_colors[opp]}{opp} team{prompt_colors['endcolor']} reached the goal, the game ends. \n \n")
              return {"endgame": True, "win": opp, "reason": "cards finished"}
          break

      else:
          guesser_history.append(f"Team {team} said: {guess}. The word was neutral.")
          if verbose: print(f"A neutral word have been selected. \n \n")
          del b[guess]
          break

  return {"endgame": False, "rc": rc, "b": b, "spyh": spymaster_history, "teamh": guesser_history}

In [69]:
# prompt: write a function to send an email to the human spymaster with the html board

import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

def send_email(recipient_email, html_board):
    """Sends an email with the HTML board to the specified recipient.

    Args:
        recipient_email: The email address of the recipient.
        html_board: The HTML content of the board.
        sender_email: Your email address (default: your_email@gmail.com).
                      **Replace with your actual email address.**
        sender_password: Your email password (default: your_password).
                         **Replace with your actual password or an app password.**
    """
    sender_email = userdata.get('codenames-mail')
    sender_password = userdata.get('codenames-mail-app')
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = recipient_email
    msg['Subject'] = "Codenames Board"

    msg.attach(MIMEText(html_board, 'html'))

    try:
        with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp:
            smtp.login(sender_email, sender_password)
            smtp.send_message(msg)
        print("Master board sent successfully!")
    except Exception as e:
        print(f"Error sending email: {e}")

## Gioco

In [79]:
#@title play game function
def play_game(lang = "eng", n_cards = 25, coloured_cards = 7, k_cards = 1, verbose = False, red_model = "gpt-4o-mini", blue_model = "gpt-4o-mini", red_agents = 3, blue_agents = 3, masterverbose = False):
  r = 1
  turn = "RED"
  board = generate_board(n = n_cards, lang = lang, c = coloured_cards, k = k_cards)
  cards_remaining = {"RED": coloured_cards + 1, "BLUE": coloured_cards}
  history = []

  if red_model == "human":
    mail = input("Insert the mail of the red spymaster:")
    send_email(mail, board4print(board, master=True))
  if blue_model == "human":
    mail = input("Insert the mail of the blue spymaster:")
    send_email(mail, board4print(board, master=True))

  if verbose:
      intro = prompt_colors["n"] + "GAME PARAMETERS:" + prompt_colors["endcolor"] + "\n" + \
      f"language = {lang}, \n" + \
      f"number of cards = {n_cards}, \n" + \
      f"number of killer cards = {k_cards}, \n" + \
      "\n" + \
      prompt_colors["RED"] + "RED TEAM PARAMETERS:" + prompt_colors["endcolor"] + "\n" + \
      "Red team starts firts, " + \
      f"number of red cards = {cards_remaining['RED']}, \n" + \
      f"red model = {red_model}, \n" + \
      f"number of red agents = {red_agents}, \n" + \
      "\n" + \
      prompt_colors["BLUE"] + "BLUE TEAM PARAMETERS:" + prompt_colors["endcolor"] + "\n" + \
      f"number of blue cards = {cards_remaining['BLUE']}, \n" + \
      f"blue model = {blue_model}, \n" + \
      f"number of blue agents = {blue_agents}, \n"

      print("-----------------------------------")
      print(intro)
      print("-----------------------------------")

  while True:
    result = play_turn(lang = lang,
                       team = turn,
                       board = board,
                       cards_remaining = cards_remaining,
                       k = k_cards,
                       n_guessers = red_agents if turn == "RED" else blue_agents,
                       history = history,
                       model = red_model if turn == "RED" else blue_model,
                       verbose = verbose,
                       masterverbose = masterverbose)
    if result["endgame"]:
      if verbose: print(result)
      return (result["win"], result["reason"], r)
    else:
      cards_remaining = result["rc"]
      board = result["b"]
      r += 1
      turn = "BLUE" if turn == "RED" else "RED"
      history.append(result["spyh"])
      history += result["teamh"]

In [97]:
play_game(verbose = True, masterverbose=True)

-----------------------------------
GAME PARAMETERS:
language = eng, 
number of cards = 25, 
number of killer cards = 1, 

RED TEAM PARAMETERS:
Red team starts firts, number of red cards = 8, 
red model = gpt-4o-mini, 
number of red agents = 3, 

BLUE TEAM PARAMETERS:
number of blue cards = 7, 
blue model = gpt-4o-mini, 
number of blue agents = 3, 

-----------------------------------
The actual board is: 


it is RED team turn: 

The RED spymaster's clue is: FLOAT (3). 
 

Team RED guesser 0 thinks: "FLOAT" makes me think of "DUCK" (floating on water) and "SINK" (the opposite). However, given we're guessing for RED, I suggest we start with "DUCK." It's a strong connection. What do you all think?.
Team RED guesser 1 thinks: For the clue "FLOAT," I'm thinking of words like "DUCK" (think of duck floats), "SINK" (opposite), and "PLANE" (it floats in the air). "DRESS" could also refer to something that flows. Let's start with "DUCK.".
Team RED guesser 2 thinks: For the clue "FLOAT," I'm thinking of words related to things that can float. "DUCK" (they float on water) and "SOLDIER" (as in soldiers on a floating boat) come to mind. Let's guess "DUCK" first!.
[True, True, True]
Team RED guesser 0 said: Let's go with 'DUCK'!.
Team RED guesser 1 said: Let's go with 'DUCK' as our guess!.
Team RED guesser 2 said: I agree, let's guess 'DUCK'!.
NARRATOR: Team RED voted: DUCK. 
 

A neutral word have bee

it is BLUE team turn: 

The BLUE spymaster's clue is: MILITARY (3). 
 

Team BLUE guesser 0 thinks: For the clue "MILITARY," I think "SOLDIER" is the most obvious choice. "FORCE" could also relate to military forces. Let's start with "SOLDIER." What do you all think?.
Team BLUE guesser 1 thinks: For the clue "MILITARY," let's focus on words related to the military. I suggest we guess "SOLDIER" first, as it's the most obvious connection. "FORCE" could be another option later. What do you all think?.
Team BLUE guesser 2 thinks: For the clue "MILITARY," I'm thinking "SOLDIER" and possibly "FORCE." "SINK" doesn't seem related. Let's focus on "SOLDIER" first. What do you all think?.
[True, True, True]
Team BLUE guesser 0 said: I agree with starting with 'SOLDIER.' Let's go with that!.
Team BLUE guesser 1 said: I think we should go with 'SOLDIER' first. Everyone seems on board with that!.
Team BLUE guesser 2 said: Let's go with 'SOLDIER' first..
NARRATOR: Team BLUE voted: SOLDIER. 
 

A BLUE

it is RED team turn: 

The RED spymaster's clue is: BANK (4). 
 

Team RED guesser 0 thinks: For the clue "BANK," I think "CENTER" could relate to a bank's central location. "SOUL" might tie to "soul bank," but it's less direct. Let's consider "TAP" as in tapping into a bank. My best guess would be "CENTER" first. What do you think?.
Team RED guesser 1 thinks: For the clue "BANK," we might consider "SINK" (like sinking money) and "POINT" (as in bank points). "CENTER" could relate to the center of a bank, but that's less direct. Let's start with "SINK.".
Team RED guesser 2 thinks: For the clue "BANK," we might think of related words like "SINK" (bank of a river), "SPOT" (bank as in a financial spot), or "POINT" (pointing to a location or a bank). Let's consider "SINK" first..
[True, True, True]
Team RED guesser 0 said: I still think 'CENTER' relates to a bank's central location. Let's discuss if we might want to try 'SINK' next..
Team RED guesser 1 said: I suggest we go with 'SINK' next

it is BLUE team turn: 

The BLUE spymaster's clue is: USA (2). 
 

Team BLUE guesser 0 thinks: Thinking about "USA," I'd consider "AMERICA" as the most direct connection. Other words like "SOUL" and "DRESS" don't seem relevant. Let's go with "AMERICA" as our first guess..
Team BLUE guesser 1 thinks: For the clue "USA," let's consider words connected to America. I suggest we start with "AMERICA" as it directly relates. What do you all think?.
Team BLUE guesser 2 thinks: For the clue "USA," I think "AMERICA" is definitely a strong candidate. "FILE" may refer to government files, but it’s less direct. I suggest we start with "AMERICA." What do you all think?.
[True, True, True]
Team BLUE guesser 0 said: I strongly believe we should go with 'AMERICA' first..
Team BLUE guesser 1 said: Let's guess 'AMERICA'. It's the most direct connection..
Team BLUE guesser 2 said: I think we should go with 'AMERICA' for our guess. Let's do it!.
NARRATOR: Team BLUE voted: AMERICA. 
 

A BLUE word have been

it is RED team turn: 

The RED spymaster's clue is: FASHION (2). 
 

Team RED guesser 0 thinks: For "FASHION," I think "DRESS" is a strong candidate since it directly relates to clothing. "ORGAN" and "SPOT" seem less relevant. Let's go with "DRESS" for our first guess. What do you all think?.
Team RED guesser 1 thinks: For the clue "FASHION," I suggest we focus on words related to clothing or style. "DRESS" stands out as a clear option. What do you all think about starting with that?.
Team RED guesser 2 thinks: For the clue "FASHION," I think "DRESS" is the most relevant. It fits perfectly with fashion. "SPOT" could refer to a brand or pattern, but less likely. Let's start with "DRESS.".
[True, True, True]
Team RED guesser 0 said: I think 'DRESS' is our best guess for FASHION. Let's go with it!.
Team RED guesser 1 said: I agree, 'DRESS' is a strong choice. Let's go with that!.
Team RED guesser 2 said: Let's go with 'DRESS' for FASHION..
NARRATOR: Team RED voted: DRESS. 
 

A RED word h

it is BLUE team turn: 

The BLUE spymaster's clue is: LIGHT (2). 
 

Team BLUE guesser 0 thinks: For the clue "LIGHT," we might consider "LASER" as it relates to light. "FILE" could refer to a way to shed light on a topic, but it's less direct. Let's go with "LASER" first!.
Team BLUE guesser 1 thinks: For the clue "LIGHT," let's consider potential connections. "LASER" stands out for its association with light beams. "SPOT" could relate to spotlights. "TAP" doesn't seem relevant. I'd suggest we guess "LASER" first. What do you think?.
Team BLUE guesser 2 thinks: For the clue "LIGHT," let's consider words associated with light or brightness. "LASER" stands out because it's a source of light. "SPOT" could relate to a spotlight. 

Let's go with "LASER" first!.
[True, True, True]
Team BLUE guesser 0 said: I agree, let's go with 'LASER' first!.
Team BLUE guesser 1 said: Let's guess 'LASER' first!.
Team BLUE guesser 2 said: Let's go with 'LASER' first; everyone agrees!.
NARRATOR: Team BLUE vo

it is RED team turn: 

The RED spymaster's clue is: DEATH (2). 
 

Team RED guesser 0 thinks: For the clue "DEATH," we could consider "UNDERTAKER" as it's directly related. "WHIP" and "BAT" might fit loosely with death themes but less directly. Let's focus on "UNDERTAKER" first. What do you all think?.
Team RED guesser 1 thinks: For the clue "DEATH," I suggest we consider words associated with finality or funerals. "UNDERTAKER" stands out strongly for me as it directly relates to death. Let's guess that first!.
Team RED guesser 2 thinks: For the clue "DEATH," I think of words related to endings or darkness. "UNDERTAKER" seems like a strong candidate. "BAT" could also reference darkness. Let's consider "UNDERTAKER" first as our guess..
[True, True, True]
Team RED guesser 0 said: I agree, let's go with 'UNDERTAKER' as our first guess..
Team RED guesser 1 said: Let's guess 'UNDERTAKER' first; it strongly relates to 'DEATH.'.
Team RED guesser 2 said: Let's go with 'UNDERTAKER' as our guess

('BLUE', 'guessed killer word', 7)